##  Covid 19 model with policy paths

<br>

__Dashboard:__ https://covid19-scenarios.netlify.com/

References:

- https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology
- https://gabgoh.github.io/COVID/index.html




### 1. Non-technical introduction

The model has several groups / compartments

- Susceptible: Individuals that are healthy and can still catch the virus. For a virus like covid 19, the whole population is assumed to start here

- Exposed: Infected individuals who are currently in a non-infectious incubation time

- Infectious incubation period: Infected individuals who are currently in the subsequent infectious part of the incubation time

Until this point, the stages have been passed by all individuals alike. After some days, the infected individuals then spread in four groups:

- Asymptomatic: Infectious inviduals who remain asymptomatic (or with very mild symptoms indistinguishable from a mild cold)

- Mild: Infectious individuals who have mild to medium symptoms

- Severe: Infectious individuals with severe symptoms. Then individuals first stay at home and then all become hospitalized

- Recovered or deceases: Individuals that recovered form the disease and are assumed to be immune. All asymptomatic and mild cases are assumed to recover, a part of the severe cases recovers and another part deceases

$R_{0}$: At the beginning of the simulation, with an all susceptible population, how many individuals does an infectious individual infect on average

<br>

### 2. Compartments

$S$ &nbsp; Susceptible

$E$ &nbsp; Non infectious incubation period (duration $t_{e-inc}$)

$I_{inc}$ &nbsp; Infectious incubation period (duration $t_{i-inc}$)

$I_{asy}$ &nbsp; Infectious, asymptomatic course (duration $t_{asy}$)

$I_{mild}$ &nbsp; Infectious, mild course (duration $t_{mild}$)

$I_{sev-pre-hos}$ &nbsp; Infectious, severe course, initial time before hospitalization (duration $t_{sev-pre-hos}$)

$I_{sev-hos-rec}$ &nbsp; Infectious, severe course, subsequent stay in hospital with recovery (duration $t_{sev-hos-dec}$)

$I_{sev-hos-dec}$ &nbsp; Infectious, severe course, subsequent stay in hospital deceased (duration $t_{sev-hos-dec}$)

$R_{asy}$ &nbsp; Recovered from asymptomatic course

$R_{mild}$ &nbsp; Recovered from mild course

$R_{sev}$ &nbsp; Recovered from severe course after hospital

$D_{sev}$ &nbsp; Deceased from severe course after hospital


________


$p_{asy}$ &nbsp; Share of individuals with asymptomatic course

$p_{sev-rec}$ &nbsp; Share of individuals with severe/hospitalized course who will recover

$p_{sev-dec}$ &nbsp; Share of individuals with severe/hospitalized course who will decease; mortality rate

-> The share of infected individuals who will go to hospital is $p_{sev-rec} + p_{sev-dec}$ 

$p_{mild} = 1 - p_{asy} - p_{sev-rec} - p_{sev-dec}$ &nbsp; Share of individuals with mild course (largest share)

$p_{icu-giv-hos}$ &nbsp; Share of those individuals in hospital assumed to be in ICU

<br>

### 3. Flowchart visualization

<img alt="Illustration of Flows" src="flow.png" width="1000"/>

### 4. Differential equations

<br>


$(1)$ &nbsp; &nbsp; $\dot{S} = \left( 1 - policy\_strength_{all} \right) (-\beta) \left(I_{inc} + I_{asy} + (1 - policy\_strength_{sym}) \; (I_{mild} + I_{sev-pre-hos})\right) S $

$(2)$ &nbsp; &nbsp; $\dot{E} = \left( 1 - policy\_strength_{all} \right) (+\beta) \left(I_{inc} + I_{asy} + (1 - policy\_strength_{sym}) \; (I_{mild} + I_{sev-pre-hos})\right) S \; - \; \frac{1}{t_{e-inc}}E$

$(3)$ &nbsp; &nbsp; $\dot{I}_{inc} = \frac{1}{t_{e-inc}}E \; - \; \frac{1}{t_{i-inc}}I$

$(4)$ &nbsp; &nbsp; $\dot{I}_{asy} = \frac{1}{t_{i-inc}}  p_{asy} I_{inc} \; - \; \frac{1}{t_{asy}}I_{asy}$

$(5)$ &nbsp; &nbsp; $\dot{I}_{mild} = \frac{1}{t_{i-inc}} p_{mild} I_{inc} \; - \; \frac{1}{t_{mild}}I_{mild}$

$(6)$ &nbsp; &nbsp; $\dot{I}_{sev-pre-hos} = \frac{1}{t_{i-inc}} (p_{sev-rec}+p_{sev-dec}) I_{inc} \; - \; \frac{1}{t_{sev-pre-hos}}I_{sev-pre-hos}$

$(7)$ &nbsp; &nbsp; $\dot{I}_{sev-hos-rec} =  \frac{1}{t_{sev-pre-hos}} \frac{p_{sev-rec}}{p_{sev-rec}+p_{sev-dec}} I_{sev-pre-hos} \; - \; \frac{1}{t_{sev-hos-rec}}I_{sev-hos-rec}$

$(8)$ &nbsp; &nbsp; $\dot{I}_{sev-hos-dec} = \frac{1}{t_{sev-pre-hos}} \frac{p_{sev-dec}}{p_{sev-rec}+p_{sev-dec}} I_{sev-pre-hos} \; - \; \frac{1}{t_{sev-hos-dec}}I_{sev-hos-dec}$

$(9)$ &nbsp; &nbsp; $\dot{R}_{asy} = \frac{1}{t_{asy}}I_{asy}$

$(10)$ &nbsp; $\dot{R}_{mild} = \frac{1}{t_{mild}}I_{mild}$

$(11)$ &nbsp; $\dot{R}_{sev} = \frac{1}{t_{sev-hos-rec}}I_{sev-hos-rec}$

$(12)$ &nbsp; $\dot{D}_{sev} = \frac{1}{t_{sev-hos-dec}}I_{sec-hos-dec}$

Lastly, we assume at any time that a constant fraction of the hospitalized cases is in the ICU: $p_{icu-giv-hos}$

<br>

### 5. Computing beta

For a given $R_{0}$, we compute the average beta for the equations above as:

$\beta = \frac{R_{0}}{t_{average}}$ &nbsp; &nbsp; with &nbsp; &nbsp; $t_{average} = t_{i-inc} + p_{asy} \cdot t_{asy} + p_{mild} \cdot t_{mild} + (p_{sev-rec}+p_{sev-dec}) \cdot t_{sev-pre-hos}$

Being infectious starts with the infectious part of the incubation period ($t_{i-inc}$) - every individual flows through this part of the model. Afterwards people are separated into groups with different illness durations. Asymptomatic and mild are assumed to be infectious throughout. The group of severe patients prior to their hospital stay is assumed to infect others too, but once in hospitals is assumed to be isolated. Thus, the hospitalization durations are not part of the average infectious period above.

<br>

### 6. Policy options

This model never uses the two policy parameters simultaneously, it only uses one of them and sets the other one to zero.

1. $policy\_strength_{all} \in [0,1]$: Percentage decrease in new infections each period resulting from a policy that affects all infectious individuals, not just the symptomatic ones (e.g. social distancing, or to some degree testing __and__ tracing). A policy strength of  0 is the baseline model, an unrealistically strong level of 100 assumes to decrease all new infections to zero.

2. $policy\_strength_{sym} \in [0,1]$: Percentage decrease in new infections each period resulting from a policy that affects only symptomatic individuals (e.g. self-quarantine of symptomatic individuals). A policy strength of  0 is the baseline model, an unrealistically strong level of 100 assumes to decrease all new infections to zero.

<br>

### 7. Computing policy implied R0

First recall, that each of the two policies is only applied individually. Then each policy has an implied $R_{0}$, with "implied" meaning to assume that the policy starts at the beginning and is sustained forever. The actual $R_{t}$ would additionally be declining for every policy as the share of susceptible individuals declines. 

1. For the policy which decreases infections caused by all individuals: $R_{0}^{implied-policy-all} = (1 - policy\_strength_{all}) \cdot \beta \cdot t_{average}$ &nbsp; &nbsp; (for a derivation of $t_{average}$ see 5.)

2. For the policy which decreases infections caused by only symptomatic individuals (now a part of the average time has to be adjusted): 


$R_{0}^{implied-policy-sym} = \beta \cdot \left[ t_{i-inc} + p_{asy} \cdot t_{asy} + (1-policy\_strength_{sym}) \left( p_{mild} \cdot t_{mild} + (p_{sev-rec}+p_{sev-dec}) \cdot t_{sev-pre-hos} \right) \right]$